# Westeros Tutorial - Introducing emissions (part1/2): Adding emission bounds

In the first part, the model chose not base the power system on wind power as electricity from  wind turbines was more expensive than electricity produced from coal. However, now we introduce emissions to investigate the impact of climate policy. Let's see what happens then.

**Pre-requisites**
- You have the *MESSAGEix* framework installed and working
- You have run Westeros baseline scenario (``westeros_baseline.ipynb``) and solved it successfully

In [1]:
import pandas as pd
import ixmp
import message_ix

from message_ix.util import make_df

%matplotlib inline

<IPython.core.display.Javascript object>

In [2]:
mp = ixmp.Platform()

In [3]:
model = 'Westeros Electrified'

base = message_ix.Scenario(mp, model=model, scenario='baseline')
scen = base.clone(model, 'emission_pool_test','introducing an upper bound on emissions',
                  keep_solution=False)
scen.check_out()

In [4]:
year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df['year_vtg'], year_df['year_act']
model_horizon = scen.set('year')
country = 'Westeros'

## Introducing Emissions

In [5]:
# first we introduce the emission of CO2 and the emission category GHG
scen.add_set('emission', 'CO2')
scen.add_cat('emission', 'GHG', 'CO2')

# we now add CO2 emissions to the coal powerplant
base_emission_factor = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'mode': 'standard',
    'unit': 'tCO2/kWa',
}

# adding new units to the model library (needed only once)
mp.add_unit('tCO2/kWa')
mp.add_unit('MtCO2')

emission_factor = make_df(base_emission_factor, technology= 'coal_ppl', emission= 'CO2', value = 7.4)
scen.add_par('emission_factor', emission_factor)

## Define a Bound on Emissions

The `type_year: cumulative` assigns an upper bound on the *weighted average of emissions* over the entire time horizon.

In [6]:
scen.add_par('bound_emission', [country, 'GHG', 'all', 'cumulative'],
             value=500., unit='MtCO2')

## Add GHG poolsize for historical period 690

In [7]:
scen.add_set('type_year', 690)
scen.add_set('cat_year', pd.DataFrame({'type_year': [690], 'year': 690}))
df = pd.DataFrame({
    'node': 'Westeros',
    'emission': 'CO2',
    'type_tec': 'all',
    'year': 690,
    'value': [100000],
    'unit': 'MtCO2'
})
scen.add_par('historical_emission_pool', df)

## Add sinkrate for model-periods

In [8]:
df = pd.DataFrame({
    'node': 'Westeros',
    'emission': 'CO2',
    'type_tec': 'all',
    'year': [700, 710, 720],
    'value': [0.02, 0.01, 0.005],
    'unit': '???'
})
scen.add_par('emission_sink_rate', df)

In [9]:
case = "carbon_pool_accounting"

## OPTIONAL: Add `tax_emission_pool`

In [10]:
add_tep = False
if add_tep:
    df = pd.DataFrame({
        'node': 'Westeros',
        'type_emission': 'GHG',
        'type_tec': 'all',
        'year': [700, 710, 720],
        'value': [0.0378193, 0, 0.04868423],
        'unit': '???'
    })
    scen.add_par('tax_emission_pool', df) 
    case = "tax_emission"

## OPTIONAL: Add `tax_emission_pool` for node `World`

In [11]:
add_tep_wrld = False
if add_tep_wrld: 
    scen.add_set('type_emission', 'CO2')
    scen.add_cat('emission', 'CO2', 'CO2')
    
    for par in ['emission_sink_rate', 'historical_emission_pool']:
        df = scen.par(par)
        df["node"] = 'World'
        scen.add_par(par, df)     
    
    df = pd.DataFrame({
        'node': 'World',
        'type_emission': 'CO2',
        'type_tec': 'all',
        'year': [700, 710, 720],
        'value': [500, 500, 500],
        'unit': '???'
    })
    scen.add_par('tax_emission_pool', df) 
    case = "tax_emission_world"

## OPTIONAL: Add `bound_emission_pool`

In [12]:
add_epb = True
if add_epb:
    df = pd.DataFrame({
        'node': 'Westeros',
        'type_emission': 'GHG',
        'type_tec': 'all',
        'year': [700, 710, 720],
        'value': [87000, 86000, 85000],
        'unit': '???'
    })
    scen.add_par('bound_emission_pool', df)
            
    # Remove bound_emission
    df = scen.par('bound_emission')
    scen.remove_par('bound_emission', df)
    case = "bound_emission"

## Time to Solve the Model

In [13]:
scen.commit(comment='introducing emissions and setting an upper bound')
scen.set_as_default()

In [14]:
scen.solve(var_list=['EMISS_POOL', 'PRICE_EMISSION_POOL'], case=case)

In [15]:
scen.var('OBJ')['lvl']

221792.71875

## Checking Results

In [16]:
#    WITHOUT any constraints EMISS_POOL should have results:
#    700 86254.747976
#    710 83581.678489
#    720 85134.173863 
    
#    WITH `tax_emission_pool' EMISS_POOL should have results:
#    700 85154.242090
#    710 78031.405954
#    720 74315.624718

#    WITH `bound_emission_pool' EMISS_POOL should have results:
#    700 87000
#    710 85975.299326
#    720 85000

In [17]:
scen.var('EMISS_POOL')

,node,emission,type_tec,year,lvl,mrg
0,Westeros,CO2,all,700,86254.747976,0.0
1,Westeros,CO2,all,710,83545.115216,0.0
2,Westeros,CO2,all,720,85000.000000,0.0


In [18]:
scen.var('PRICE_EMISSION_POOL')

,node,type_emission,type_tec,year,lvl,mrg
0,Westeros,GHG,all,720,33.075572,0.0


## Close the connection to the database

In [19]:
mp.close_db()